
<h1 style="text-align: center;">CIÊNCIA DE DADOS</h1>
<h1 style="text-align: center;">Roteiro de Atividade Prática</h1>
<br>
<br>

Nome: ______________________________________________________________________________________      

Turma: ______________


**Componente:** Inteligência Artificial
<br>
**Unidade Curricular:** IA Generativa
<br>
**Tema da Semana:** Implantação de LLMs
<br>


# Aula 3: Exercícios de fixação

## Tarefa
- Executem o código abaixo.

- Se as bibliotecas não tiverem instaladas, instale-as.

- Acompanhem as instruções.

- Observem a execução e discutam com seus colegas e professor.


### Atualizar bibliotecas

In [ ]:
!pip install -U langchain langchain-huggingface transformers faiss-cpu

### Importação das bibliotecas necessárias

In [ ]:
import os
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFacePipeline
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import pipeline


### Criar embeddings com modelo leve


In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


### Criar uma base de conhecimento com nome de documento.txt
**Neste exemplo iremos utilizar o texto em inglês, para isso utilize o google tradutor para adicionar mais informações**

In [ ]:


if not os.path.exists("dados"):
    os.makedirs("dados")

with open("dados/documento_aula3.txt", "w", encoding="utf-8") as f:
    f.write("XYZ Platform Terms of Use and Security\n\n"
        "1. **Use of the Platform:** The user agrees to use the services of the XYZ Platform only for " "lawful purposes and in accordance with the established terms. Any attempt at fraudulent use will result in " "immediate blocking of the account.\n\n"
        "2. **Data Privacy:** All information collected is stored securely and is never " "shared with third parties without the user's consent. The data is protected by encryption "
        "and follows the standards of the General Data Protection Law (LGPD).\n\n"
        "3. **Information Security:** The XYZ Platform implements two-factor authentication (2FA) to "
        "protect access to accounts. We recommend that users enable this functionality to increase "
        "security.\n\n"
        "4. **User Rights:** Users have the right to request the deletion of their data at any "
        "time. To do so, simply contact us by email at suporte@xyz.com.\n\n"
        "5. **Updates to the Terms:** The XYZ Platform may update its terms of use periodically. "
        "Users will be notified of relevant changes via email or directly on the platform.\n")


### Criar banco de vetores FAISS

In [ ]:
loader = TextLoader("dados/documento_aula3.txt")
documentos = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documentos_divididos = text_splitter.split_documents(documentos)
vectorstore = FAISS.from_documents(documentos_divididos, embeddings)
retriever = vectorstore.as_retriever()


### Criar modelo LLM leve

In [ ]:

llm_pipeline = pipeline("text-generation", model="gpt2", max_new_tokens=100, temperature=0.7, pad_token_id=50256)
llm = HuggingFacePipeline(pipeline=llm_pipeline)


### Criar a cadeia de recuperação e resposta


In [ ]:
qa = RetrievalQA.from_llm(llm=llm, retriever=retriever)


### Teste do sistema com RAG com multiplas perguntas


In [ ]:
perguntas = [
    "What are the rights of users on the XYZ Platform?",
    "Does the XYZ Platform share my data with third parties?",
    "How can I protect my account?",
    "What happens if I violate the terms of use?"
]

for pergunta in perguntas:
    print("\n================================================\n")
    resposta = qa.invoke({"query": pergunta})
    print(f"\n Pergunta: {pergunta}")
    print(f"\nPergunta/Query:\n", resposta['query'])
    print(f"\nResultado:\n", resposta['result'])
